In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import key

# Configure gmaps
gmaps.configure(api_key=key)

In [ ]:
df = pd.read_excel("Compiled_Nashville_September_Property_Sales_GetCoordsFinal.xlsx")
df['Sale Price'] = df['Sale Price'].apply(lambda x: "${:.1f}k".format((x/1000)))
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,Zone,Coordinates
0,053 05 0A 008.00,SINGLE FAMILY,1100 CHEYENNE CIR,MADISON,2021-09-07,$304.0k,20210921-0126315,No,No,6,"36.2524142, -86.6706970"
1,006 00 0 010.00,SINGLE FAMILY,3450 GREER RD,GOODLETTSVILLE,2021-09-17,$405.0k,20210920-0125863,No,No,5,"36.3835361, -86.7832459"
2,004 00 0 075.00,SINGLE FAMILY,8565 WHITES CREEK PIKE,JOELTON,2021-09-22,$666.5k,20211001-0131984,No,No,5,"36.3811685, -86.8945705"
3,004 00 0 193.00,SINGLE FAMILY,8560 COOPERTOWN RD,JOELTON,2021-09-20,$620.0k,20210924-0128377,No,No,5,"36.3808100, -86.9061216"
4,004 00 0 139.00,SINGLE FAMILY,8409 RILEY ADCOCK RD,JOELTON,2021-09-17,$227.5k,20210923-0127923,No,No,5,"36.3729140, -86.8836440"


In [46]:
df[["Latitude","Longitude"]] = df["Coordinates"].str.split(",", expand = True)
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,Zone,Coordinates,Latitude,Longitude
0,053 05 0A 008.00,SINGLE FAMILY,1100 CHEYENNE CIR,MADISON,2021-09-07,$304.0k,20210921-0126315,No,No,6,"36.2524142, -86.6706970",36.2524142,-86.6706970
1,006 00 0 010.00,SINGLE FAMILY,3450 GREER RD,GOODLETTSVILLE,2021-09-17,$405.0k,20210920-0125863,No,No,5,"36.3835361, -86.7832459",36.3835361,-86.7832459
2,004 00 0 075.00,SINGLE FAMILY,8565 WHITES CREEK PIKE,JOELTON,2021-09-22,$666.5k,20211001-0131984,No,No,5,"36.3811685, -86.8945705",36.3811685,-86.8945705
3,004 00 0 193.00,SINGLE FAMILY,8560 COOPERTOWN RD,JOELTON,2021-09-20,$620.0k,20210924-0128377,No,No,5,"36.3808100, -86.9061216",36.3808100,-86.9061216
4,004 00 0 139.00,SINGLE FAMILY,8409 RILEY ADCOCK RD,JOELTON,2021-09-17,$227.5k,20210923-0127923,No,No,5,"36.3729140, -86.8836440",36.3729140,-86.8836440


In [47]:
# Create DataFrame called
fast_food_df = df[["Property Address", "Property City", "Sale Price", "Latitude", "Longitude"]].copy()
fast_food_df["Fast Food"] = ""
fast_food_df.head(10)

,Property Address,Property City,Sale Price,Latitude,Longitude,Fast Food
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.2524142,-86.6706970,
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.3835361,-86.7832459,
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.3811685,-86.8945705,
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.3808100,-86.9061216,
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.3729140,-86.8836440,
5,7840 WHITES CREEK PIKE,JOELTON,$575.0k,36.3553598,-86.8780549,
6,3560 BAXTER RD,JOELTON,$805.0k,36.3527869,-86.8850078,
7,7564 BIDWELL RD,JOELTON,$313.5k,36.3436573,-86.8932319,
8,208 A SWIFT DR,GOODLETTSVILLE,$333.0k,36.3354895,-86.7203211,
9,109 SWIFT DR,GOODLETTSVILLE,$350.0k,36.3345218,-86.7194276,


In [44]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params  = {
    'type': 'resturant',
    'radius': 1600,
    'key': key
    }

def get_find_fast_food{
    
}

KeyboardInterrupt: 